In [61]:
import numpy as np
import pandas as pd

In [62]:
a= pd.read_pickle(fr'Q:\sachuriga\CR_CA1_paper/tables/all_units_table_with_waveforms.pkl')
pairs_df=pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/Remapping_ratemaps_unsmoothed.pkl")
# Assuming your data is loaded in pairs_df

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, mannwhitneyu
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.stats import ttest_ind


fig = plt.figure(figsize=(7.2, 6),dpi=1200)
plt.rcParams.update({'font.size': 8,'font.family': 'DejaVu Sans'})
gs = gridspec.GridSpec(4, 6, height_ratios=[.4, .4, 0.6, 0.6], width_ratios=[1,1,1,1,1,1])  # First row taller
plt.rcParams.update({
    'axes.labelpad': -0.1,
    'ytick.major.pad': -0.1,
    'xtick.major.pad': -0.1,
    'ytick.major.size': 2,
    'xtick.major.size': 2
})

ax1_2 = fig.add_subplot(gs[0:2, 0:2])
ax1_3 = fig.add_subplot(gs[0:2, 2:4])
ax1_4 = fig.add_subplot(gs[0:2, 4:6])


ax2_1 = fig.add_subplot(gs[2, 0])
ax2_2 = fig.add_subplot(gs[2, 1])
ax2_3 = fig.add_subplot(gs[2, 2])
ax2_4 = fig.add_subplot(gs[2, 3])
ax2_5 = fig.add_subplot(gs[2, 4])
ax2_6 = fig.add_subplot(gs[2, 5])

ax3_1 = fig.add_subplot(gs[3, 0])
ax3_2 = fig.add_subplot(gs[3, 1])
ax3_3 = fig.add_subplot(gs[3, 2])
ax3_4 = fig.add_subplot(gs[3, 3])
ax3_5 = fig.add_subplot(gs[3, 4])
ax3_6 = fig.add_subplot(gs[3, 5])


# ax2_1 = fig.add_subplot(gs[1, 0], projection='3d')
# ax2_2 = fig.add_subplot(gs[1, 1])
# ax2_3 = fig.add_subplot(gs[1, 2])
# ax2_4 = fig.add_subplot(gs[1, 3])

pairs_df=pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/Remapping_ratemaps_unsmoothed.pkl")
# Assuming your data is loaded in pairs_df
df = pairs_df[(pairs_df['cell_type1']=="pyramidal") 
              & (pairs_df['cell_type2']=="pyramidal")]


# df = pairs_df[(pairs_df['neuron location 1']=="superficial") 
#               & (pairs_df['neuron location 2']=="superficial")]


# Define control and experimental animal_ids
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']

# Split into three groups based on Session1 and Session2
ab_group = df[(df['Session1'] == 1.0) & (df['Session2'] == 2.0)]  # AB group
ba_group = df[(df['Session1'] == 2.0) & (df['Session2'] == 3.0)]  # BA' group
aa_group = df[(df['Session1'] == 1.0) & (df['Session2'] == 3.0)]  # AA' group

# Split into control and experimental groups
control_ab = ab_group[ab_group['animal_id'].isin(control_ids)]
control_ba = ba_group[ba_group['animal_id'].isin(control_ids)]
control_aa = aa_group[aa_group['animal_id'].isin(control_ids)]

exp_ab = ab_group[ab_group['animal_id'].isin(exp_ids)]
exp_ba = ba_group[ba_group['animal_id'].isin(exp_ids)]
exp_aa = aa_group[aa_group['animal_id'].isin(exp_ids)]

# Prepare plotting data
plot_data = []

# Add control group data
for group_num, group_df in enumerate([control_ab, control_ba, control_aa], 1):
    for corr_value in group_df['Corr']:
        plot_data.append({
            'Corr': corr_value,
            'Group': f'Group{group_num}',
            'Condition': 'Control'
        })

# Add experimental group data
for group_num, group_df in enumerate([exp_ab, exp_ba, exp_aa], 1):
    for corr_value in group_df['Corr']:
        plot_data.append({
            'Corr': corr_value,
            'Group': f'Group{group_num}',
            'Condition': 'Experimental'
        })

# Create DataFrame for plotting
plot_df = pd.DataFrame(plot_data)

axes_cdf = [ax1_2,ax1_3,ax1_4]

group_labels = ['S1=1.0, S2=2.0', 'S1=2.0, S2=3.0', 'S1=1.0, S2=3.0']
alternatives = ['two-sided', 'two-sided', 'two-sided']  # AB: Exp > Control, BA': two-sided, AA': Exp < Control

# Perform statistical tests and create plots
for i, group in enumerate(['Group1', 'Group2', 'Group3']):
    # Get data for current group
    control_data = plot_df[(plot_df['Group'] == group) & (plot_df['Condition'] == 'Control')]['Corr']
    exp_data = plot_df[(plot_df['Group'] == group) & (plot_df['Condition'] == 'Experimental')]['Corr']
    
    # Perform KS test
    ks_statistic, ks_p_value = ks_2samp(control_data, exp_data, alternative=alternatives[i])
    # Perform Mann-Whitney U test
    mw_statistic, mw_p_value = ttest_ind(control_data, exp_data, alternative=alternatives[i])
    print(f"Group{i+1} ({group_labels[i]}): KS p-value = {ks_p_value:.4f}, MW p-value = {mw_p_value:.4f}")
    # Plot CDF (second row)
    sns.ecdfplot(data=control_data, ax=axes_cdf[i], color='blue', label='Control')
    sns.ecdfplot(data=exp_data, ax=axes_cdf[i], color='red', label='Experimental')
    

    # Customize CDF subplot
    #axes_cdf[i].set_title(f'Mann-Whitney p = {mw_p_value:.3f}', fontsize=10)
    axes_cdf[i].set_xlabel('Spatial correlation')
    axes_cdf[i].set_ylabel('% Neuron')
    axes_cdf[i].legend()
    axes_cdf[i].grid(False)
    axes_cdf[i].set_aspect('auto')
    axes_cdf[i].spines['right'].set_visible(False)
    axes_cdf[i].spines['top'].set_visible(False)
    axes_cdf[i].legend().set_visible(False)
    axes_cdf[i].set_xlim([-0.4, 1])

        # Calculate means and SEM
    control_mean = np.mean(control_data)
    exp_mean = np.mean(exp_data)
    control_sem = stats.sem(control_data)
    exp_sem = stats.sem(exp_data)

    # Create inset axes for bar plot in top-right corner
    inset_ax = inset_axes(axes_cdf[i], width="80%", height="80%", loc='lower right',
                     bbox_to_anchor=(0.7, 0, 0.3, 0.3),  # 4-tuple: x0, y0, width, height
                     bbox_transform=axes_cdf[i].transAxes)
    
    inset_ax.bar([0.2, 0.8], [control_mean, exp_mean], yerr=[control_sem, exp_sem], 
                 color=['blue', 'red'], alpha=0.9, width=0.45, capsize=2)
    inset_ax.set_xticks([])
    inset_ax.set_yticks([])
    #inset_ax.set_xticklabels(['CRs +', 'CRs -'], fontsize=8)
    #inset_ax.set_ylabel('Mean', fontsize=8)
    inset_ax.tick_params(axis='both', labelsize=6)
    inset_ax.spines['top'].set_visible(False)
    inset_ax.spines['right'].set_visible(False)
    inset_ax.spines['left'].set_visible(False)
    inset_ax.set_ylim([0, 0.6])


w1_num = [0,9,16,17,18,39]
w2_num = [40,44,48,90,94,162]
axes_w1 = [ax2_1,ax2_2,ax2_3,ax2_4,ax2_5,ax2_6]
axes_w2 = [ax3_1,ax3_2,ax3_3,ax3_4,ax3_5,ax3_6]
df = pairs_df[(pairs_df['cell_type1']=="pyramidal") 
              & (pairs_df['cell_type2']=="pyramidal")]

i=0
for num in w1_num:
        #axes_cdf[i].set_title(f'Mann-Whitney p = {mw_p_value:.3f}', fontsize=10)
    #axes_w1[i].set_xlabel('Spatial correlation')
    #axes_w1[i].set_ylabel('% Neuron')
    axes_w1[i].plot(df['w1'].iloc[num], label='Waveform', color='black',linewidth = 1)
    axes_w1[i].plot(df['w2'].iloc[num] + 200, label='Waveform', color='grey',linewidth = 1)
    axes_w1[i].legend()
    axes_w1[i].grid(False)
    axes_w1[i].set_aspect('auto')
    axes_w1[i].spines['right'].set_visible(False)
    axes_w1[i].spines['top'].set_visible(False)
    axes_w1[i].spines['left'].set_visible(False)
    axes_w1[i].spines['bottom'].set_visible(False)
    axes_w1[i].legend().set_visible(False)
    axes_w1[i].set_xticks([])  # Hide x-axis ticks and labels
    axes_w1[i].set_yticks([])  # Hide y-axis ticks and labels
    axes_w1[i].set_ylim([-1500,1000])
    if i==0:
        # Draw a specific horizontal line at y=5, from x=0 to x=10
        axes_w1[i].hlines(y=-200, xmin=60, xmax=90, color='black', label='Horizontal line at y=5')

        # Draw a specific vertical line at x=3, from y=0 to y=10
        axes_w1[i].vlines(x=60, ymin=-200, ymax=-700, color='grey', label='Vertical line at x=3')
    i+=1


i=0
for num in w2_num:
    axes_w2[i].plot(df['w1'].iloc[num], label='Waveform', color='black',linewidth = 1)
    axes_w2[i].plot(df['w2'].iloc[num] + 200, label='Waveform', color='grey',linewidth = 1)

    axes_w2[i].legend()
    axes_w2[i].grid(False)
    axes_w2[i].set_aspect('auto')
    axes_w2[i].spines['right'].set_visible(False)
    axes_w2[i].spines['top'].set_visible(False)
    axes_w1[i].spines['top'].set_visible(False)
    axes_w2[i].spines['left'].set_visible(False)
    axes_w2[i].spines['bottom'].set_visible(False)
    axes_w2[i].legend().set_visible(False)
    axes_w2[i].set_xticks([])  # Hide x-axis ticks and labels
    axes_w2[i].set_yticks([])  # Hide y-axis ticks and labels
    axes_w2[i].set_ylim([-1500,1000])
    i+=1


plt.tight_layout()
fig.subplots_adjust(top=0.92, bottom=0.08, left=0.05, right=0.95, hspace=0.8, wspace=0.8)
plt.savefig(r'Q:/sachuriga/CR_CA1_paper/Figures/suppfig6.png', transparent=True, dpi=1200, bbox_inches='tight')

Group1 (S1=1.0, S2=2.0): KS p-value = 0.9836, MW p-value = 0.8040
Group2 (S1=2.0, S2=3.0): KS p-value = 0.2924, MW p-value = 0.5093
Group3 (S1=1.0, S2=3.0): KS p-value = 0.2326, MW p-value = 0.0807


C:\Users\sachur\AppData\Local\Temp\ipykernel_21656\1238132704.py:213: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plotting the waveform
plt.figure(figsize=(8, 4))
plt.plot(a['waveform'][0], label='Waveform', color='blue')
plt.title('Waveform Plot')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()
plt.show()